# Structure From Motion

### Vincent Hock, Tomas Arevalo, Callum Taylor

In [1]:
# Mount your google drive. This will launch a pop-up window for authentication.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Adjust this to your directory
%cd "/content/drive/My Drive/es-143-assignments/Project/"
# %cd "/content/drive/MyDrive/Harvard/ES143/Project/"

/content/drive/My Drive/es-143-assignments/Project


In [3]:
%pip install pupil-apriltags

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.7 MB/s eta 0:00:00


In [4]:
# Import any required libraries here
import cv2                   # OpenCV
import os
import numpy as np                       # numpy
from pupil_apriltags import Detector
import requests
import pickle
import glob   # filename and path management for file I/O
import scipy.interpolate  # interpolation
from pathlib import Path

# Use this line to make matplotlib plot inline (only need to call it once when importing matplotlib)
%matplotlib inline

import matplotlib.pyplot as plt
import plotly.graph_objects as go
# Modify this line to adjust the displayed plot size. You can also call
# it with different parameters before specific plots.
plt.rcParams['figure.figsize'] = [10, 10]

In [5]:
#@title helper functions: `in2hom()`, `hom2in()`, `normalizing_transform()`

def in2hom(X):
    """
    Convert from inhomogeneous to homogeneous coordinates

    Args:
      X  - NxD numpy array, tyically with D=2 (rows (x,y)) or D=3 (rows (x,y,z))

    Returns:
      Xh - Nx(D+1) numpy array with an appended column of ones
    """

    return np.concatenate([X, np.ones((X.shape[0], 1), dtype=np.float32)], axis=1)

# Convert from homogeneous to inhomogeneous coordinates
def hom2in(X):
    """
    Convert from homogeneous to inhomogeneous coordinates

    Args:
      Xh  - Nx(D+1) numpy array, tyically with D=2 (rows (x,y,w)) or D=3 (rows (x,y,z,w))

    Returns:
      X - NxD numpy array with the first D columns divided by column (D+1)
    """

    return X[:, :-1] / X[:, -1:]

def normalizing_transform(X):
    """
    Compute a (Dx1)x(Dx1) normalizing transformation from N points in D dimensions

    Args:
      X - NxD numpy array, tyically with D=2 (rows (x,y)) or D=3 (rows (x,y,z))

    Returns:
      T - a (D+1)x(D+1) matrix that normalizes the points to be centered at their
          centroid, with average distance equal to sqrt(D)
    """

    # dimension of datapoints
    D = X.shape[1]

    # Compute centroid
    centroid = X.mean(axis=0, keepdims=True)

    # Compute the denominator of scale factor s
    denom = np.mean(np.sqrt(np.sum((X - centroid) ** 2, axis=1)))

    # Compute s, tx, ty
    s = np.sqrt(D) / denom
    t = -s * centroid[0, :]
    t = np.concatenate((t,np.array([1])))

    # return matrix T
    return np.concatenate((s*np.eye(D+1,D+1)[:,0:D], np.expand_dims(t,1)),axis=1)


#Camera Calibration

Note: this requires camera to be calibrated using AprilBoards

In [6]:
#@title Get aprilboard data
response = requests.get("https://github.com/Harvard-CS283/pset-data/raw/f1a90573ae88cd530a3df3cd0cea71aa2363b1b3/april/AprilBoards.pickle")
data = pickle.loads(response.content)

at_coarseboard = data['at_coarseboard']
at_fineboard = data['at_fineboard']

In [7]:
#@title `detect_aprilboard()`
# set up april tag detector (I use default parameters; seems to be OK)
at_detector = Detector(families='tag36h11',
                       nthreads=1,
                       quad_decimate=1.0,
                       quad_sigma=0.0,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)

def detect_aprilboard(img, board, apriltag_detector):
    """
    Detects April tags in a grayscale image.

    Usage: imgpoints, objpoints, tag_ids = detect_aprilboard(img, board, AT_detector)

    Input:
        image -- grayscale image
        board -- at_coarseboard or at_fineboard (list of dictionaries)
        AT_detector -- AprilTag Detector parameters

    Returns:
        imgpoints -- Nx2 numpy array of (x,y) image coords
        objpoints -- Nx3 numpy array of (X,Y,Z=0) board coordinates (in inches)
        tag_ids -- Nx1 list of tag IDs
    """

    imgpoints=[]
    objpoints=[]
    tagIDs=[]

    # detect april tags
    imgtags = apriltag_detector.detect(img,
                                    estimate_tag_pose=False,
                                    camera_params=None,
                                    tag_size=None)

    if len(imgtags):
        # collect image coordinates of tag centers
        # imgpoints = np.vstack([ sub.center for sub in tags ])

        # list of all tag_id's that are in board
        brdtagIDs = [ sub['tag_id'] for sub in board ]

        # list of all detected tag_id's that are in image
        imgtagIDs = [ sub.tag_id for sub in imgtags ]

        # list of all tag_id's that are in both
        tagIDs = list(set(brdtagIDs).intersection(imgtagIDs))

        if len(tagIDs):
            # all board list-elements that contain one of the common tag_ids
            objs=list(filter(lambda tagnum: tagnum['tag_id'] in tagIDs, board))

            # their centers
            objpoints = np.vstack([ sub['center'] for sub in objs ])

            # all image list-elements that contain one of the detected tag_ids
            imgs=list(filter(lambda tagnum: tagnum.tag_id in tagIDs, imgtags))

            # their centers
            imgpoints = np.vstack([ sub.center for sub in imgs ])

    return imgpoints, objpoints, tagIDs

In [8]:
#@title `calibrate_camera()`

def calibrate_camera(image_folder):
  # detect fiducials
  N = 70 # only use images with at least N detected objects for calibration
  total_valid = 0

  # Edit this line to point to the collection of input calibration image
  CALIBFILES = f'./data/{image_folder}/*.JPG'

  # Uncomment one of the following two lines to indicate which AprilBoard is being used (fine or coarse)
  BOARD = at_fineboard
  #BOARD = at_coarseboard

  ###### BEGIN CALIBRATION SCRIPT

  # exit if no images are found or if BOARD is unrecognized
  images = glob.glob(CALIBFILES)
  assert images, "no calibration images matching: " + CALIBFILES
  assert BOARD==at_fineboard or BOARD==at_coarseboard, "Unrecognized AprilBoard"

  # else continue
  print("{} images:".format(len(images)))

  # initialize 3D object points and 2D image points
  calObjPoints = []
  calImgPoints = []

  # loop through the images
  for count,fname in enumerate(images):

      # read image and convert to grayscale if necessary
      orig = cv2.imread(fname)
      if len(orig.shape) == 3:
          img = cv2.cvtColor(orig, cv2.COLOR_RGB2GRAY)
      else:
          img = orig

      # detect apriltags and report number of detections
      imgpoints, objpoints, tagIDs = detect_aprilboard(img,BOARD,at_detector)
      print(f"{count} {fname}: {len(imgpoints)} imgpts, {len(objpoints)} objpts")

      # append detections if some are found
      if len(imgpoints) >= N and len(objpoints) >= N:
          total_valid += 1

          # append points detected in all images, (there is only one image now)
          calObjPoints.append(objpoints.astype('float32'))
          calImgPoints.append(imgpoints.astype('float32'))

  # calibrate the camera
  reprojerr, calMatrix, distCoeffs, calRotations, calTranslations = cv2.calibrateCamera(
      calObjPoints,
      calImgPoints,
      img.shape,    # uses image H,W to initialize the principal point to (H/2,W/2)
      None,         # no initial guess for the remaining entries of calMatrix
      None,         # initial guesses for distortion coefficients are all 0
      flags = None) # default contstraints (see documentation)

  return reprojerr, calMatrix, distCoeffs

In [9]:
# Replace input with directory of calibration images
reprojerr, calMatrix, distCoeffs = calibrate_camera("playroom_lowres_images")

24 images:
0 ./data/playroom_lowres_images/playroom_es143_small_00001.JPG: 55 imgpts, 55 objpts
1 ./data/playroom_lowres_images/playroom_es143_small_00024.JPG: 3 imgpts, 3 objpts
2 ./data/playroom_lowres_images/playroom_es143_small_00023.JPG: 22 imgpts, 22 objpts
3 ./data/playroom_lowres_images/playroom_es143_small_00019.JPG: 74 imgpts, 74 objpts
4 ./data/playroom_lowres_images/playroom_es143_small_00022.JPG: 64 imgpts, 64 objpts
5 ./data/playroom_lowres_images/playroom_es143_small_00012.JPG: 28 imgpts, 28 objpts
6 ./data/playroom_lowres_images/playroom_es143_small_00011.JPG: 51 imgpts, 51 objpts
7 ./data/playroom_lowres_images/playroom_es143_small_00017.JPG: 73 imgpts, 73 objpts
8 ./data/playroom_lowres_images/playroom_es143_small_00021.JPG: 71 imgpts, 71 objpts
9 ./data/playroom_lowres_images/playroom_es143_small_00014.JPG: 12 imgpts, 12 objpts
10 ./data/playroom_lowres_images/playroom_es143_small_00013.JPG: 9 imgpts, 9 objpts
11 ./data/playroom_lowres_images/playroom_es143_small_000

# Keypoint Detection & Matching

In [10]:
# Create SIFT object for keypoint matching
sift = cv2.SIFT_create()

In [11]:
#@title `detectKeypoints`: returns `keypoints`, `descriptors` for a single image
def detectKeypoints(im):
    # Use the detectAndCompute method of this SIFT object to
    #  detect SIFT keypoints and compute descriptor for each one
    keypoints, descriptors = sift.detectAndCompute(im, None)

    return keypoints, descriptors

In [12]:
#@title `detectKeypoints`: returns the best `GOOD_MATCH_PERCENTAGE` of the matches between two images
def matchKeypoints(keypoints1, keypoints2, descriptors1, descriptors2, percentage):
    """
    Find interest-point matches between two lists of interest point descriptors

    Args:
      keypoints1 - keypoints from sift.detectAndCompute() applied to Image 1
      keypoints2 - keypoints from sift.detectAndCompute() applied to Image 2
      descriptors1 - descriptors from sift.detectAndCompute() applied to Image 1
      descriptors2 - descriptors from sift.detectAndCompute() applied to Image 2
      percentage - number between 0 and 1

    Returns
      matches - list of Dmatch objects, which each have the following atrributes:
                DMatch.distance - Distance between descriptors. Lower is better.
                DMatch.trainIdx - Index of descriptor in "train" descriptors (descriptors1)
                DMatch.queryIdx - Index of descriptor in "query" descriptors (descriptors2)
                DMatch.imgIdx   - Index of the train image (useful when working with a large set of images; can be ignored here)
      X1 - Nx2 numpy array of points (x,y) from Image 1
      X1 - Nx2 numpy array of points (x,y) from Image 2
    """

    # Create a "brute force" matcher object using the L2 norm,
    #  and using "cross check" to only keep symmetric matches
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Use the match method of this matcher object to match
    #   descriptors between images
    matches12 = bf.match(descriptors1, descriptors2)
    num_matches12 = len(matches12)

    # TO DO: modify this line of code to find and return the
    #   (percentage * num_matches12) subset of matches that have
    #   the smallest distance between them

    goodmatches12 = sorted(matches12, key = lambda x:x.distance)[:int(percentage * num_matches12)]

    # for convenience, extract and return pixel coordinates of the good matches
    X1 = np.array([keypoints1[match.queryIdx].pt for match in matches12])
    X2 = np.array([keypoints2[match.trainIdx].pt for match in matches12])

    return goodmatches12, X1, X2

In [13]:
GOOD_MATCH_PERCENTAGE = 0.2

# Base View
Construct a point cloud from the first two images

In [96]:
def find_colors(im, points, region_size=2):
    colors = []
    half_region = region_size // 2  # Calculate half the region size for easy indexing

    for (x, y) in points.astype(np.int64):
        # Ensure the region stays within the image boundaries
        x_start = max(x - half_region, 0)
        y_start = max(y - half_region, 0)
        x_end = min(x + half_region + 1, im.shape[1])
        y_end = min(y + half_region + 1, im.shape[0])

        # Extract the region
        region = im[y_start:y_end, x_start:x_end]

        # Calculate the average color in this region
        average_color = np.mean(region, axis=(0, 1)).astype(int)

        # Append the average color to the list
        colors.append(average_color)

    return np.array(colors)

In [97]:
# Load and process base images (first two views)
im1 = cv2.cvtColor(cv2.imread('./data/p1.jpg'), cv2.COLOR_BGR2RGB)
keypoints1, descriptors1 = detectKeypoints(im1)

im2 = cv2.cvtColor(cv2.imread('./data/p2.jpg'), cv2.COLOR_BGR2RGB)
keypoints2, descriptors2 = detectKeypoints(im2)

# Find matches between base images
_, X1, X2 = matchKeypoints(keypoints1, keypoints2, descriptors1, descriptors2, GOOD_MATCH_PERCENTAGE)

# Calculate rotation and translation between the two base views
E, base_inliers = cv2.findEssentialMat(X1, X2, calMatrix, method=cv2.RANSAC)
R1, R2, t = cv2.decomposeEssentialMat(E)

# Create RANSAC inlier mask for valid matches
base_inliers = base_inliers.ravel()==1
X1 = X1[base_inliers]
X2 = X2[base_inliers]

# Calculate the camera matrix P for both views (first camera located at world coordinate origin)
P1 = calMatrix @ np.hstack((np.eye(3), np.zeros((3, 1))))
P2 = calMatrix @ np.hstack((-R2, t))

# Triangulate matches to create 3D object point cloud
base_points = cv2.triangulatePoints(P1, P2, X1.T, X2.T)
base_points = hom2in(base_points.T)

# Find colors in the base images
total_colors = find_colors(im2,X2)

# Adding Additional Views

In [98]:
#@title Define iterative variables

# 2D points in inlier matches from the previous image
X_prev_total = X2

# 3D points corresponding to 2D points in X_prev_total
prev_3d_points = base_points

# Combination of all point clouds
total_3d_points = base_points

# All keypoints & descriptors from the previous image
keypoints_prev, descriptors_prev = keypoints2, descriptors2

# Previous cameras
P_prev = P2
cameras = [P1, P2]

In [99]:
start = 3 # First image to use
end = 9 # Last image to use

for i in range(start, end+1):
  # Load and process next image
  im_curr = cv2.cvtColor(cv2.imread(f'./data/p{i}.jpg'), cv2.COLOR_BGR2RGB)
  keypoints_curr, descriptors_curr = detectKeypoints(im_curr)

  # Calculate matches between new and previous keypoints
  _, X_prev_matches, X_curr_matches = matchKeypoints(keypoints_prev, keypoints_curr, descriptors_prev, descriptors_curr, GOOD_MATCH_PERCENTAGE)

  # Select RANSAC inliers
  _, inliers = cv2.findEssentialMat(X_prev_matches, X_curr_matches, calMatrix, method=cv2.RANSAC)
  inliers = inliers.ravel()==1
  X_prev_matches = X_prev_matches[inliers]
  X_curr_matches = X_curr_matches[inliers]

  # Find colors in the current image
  total_colors = np.concatenate([total_colors, find_colors(im_curr,X_curr_matches)])

  # Find the subset of new keypoints that have corresponding matches in the previous image
  correspondence_mask = np.array([True if pt in X_prev_total else False for pt in X_prev_matches])
  X_curr_submatches = X_curr_matches[correspondence_mask]
  X_prev_submatches = X_prev_matches[correspondence_mask]

  # Find the 3D correspondences for all the 2D points selected by the correspondence mask (BRUTE FORCE)
  correspondences_3d = []
  for sub_point in X_prev_submatches:
    match_found = False
    for pt3d, tot_point in zip(prev_3d_points, X_prev_total):
      # Check if point is a match
      if tot_point[0] == sub_point[0] and tot_point[1] == sub_point[1]:
        correspondences_3d.append(pt3d)
        match_found = True
        break
    # This can be modified to remove the 2D point if no 3D point is found (technically shouldn't happen)
    if not match_found:
      print(f"Error in p{i}.jpg")

  correspondences_3d = np.array(correspondences_3d)

  # New camera pose estimation using 3D-2D correspondences calculated above
  _, rvec, t, _ = cv2.solvePnPRansac(correspondences_3d, X_curr_submatches, calMatrix, None)
  R, _ = cv2.Rodrigues(rvec)
  P_curr = calMatrix @ np.hstack((R, t))

  # Triangulate all matches using new estimated camera pose and add them to existing 3D points
  curr_3d_points = cv2.triangulatePoints(P_prev, P_curr, X_prev_matches.T, X_curr_matches.T)
  curr_3d_points = hom2in(curr_3d_points.T)
  total_3d_points = np.concatenate([total_3d_points, curr_3d_points], axis=0)

  # Store current variables for next iteration
  prev_3d_points = curr_3d_points
  keypoints_prev, descriptors_prev = keypoints_curr, descriptors_curr
  X_prev_total = X_curr_matches
  cameras.append(P_curr)
  P_prev = P_curr

# Visualize

In [100]:
#@title `add_plotly_camera`: helper function to plot camera position and direction
def add_plotly_camera(h,w,camera,raysize,figobj):
# Add tetrahedral camera to pyplot figure
# h,w:      height and width of image in pixels
# camera:   3x4 camera matrix
# raysize:  length of tetrahedral edges (in world units)
# fig:      pyplot figure object
#
# Returns: 1
#
# Uses anatomy of camera matrices from Hartley and Zisserman Chapter 6

    # normalize camera such that bottom-left three-vector
    #   corresponds to unit-length principal ray in front of camera (HZ Section 6.2.3)
    camera=camera*np.sign(np.linalg.det(camera[:,0:3]))/np.linalg.norm(camera[2,0:3])

    # Compute camera center (null vector of P)
    _, _, v = np.linalg.svd(camera)
    C = np.transpose(v[-1,0:3]) / v[-1,3]

    # Back-project image corners to unit-length 3D ray segments:
    S = np.array([[0, 0, 1],       # homog image coords if top left pixel
                  [0, h-1, 1],     # bottom left
                  [w-1, h-1, 1],   # bottom right
                  [w, 0, 1]])      # top right

    #   HZ equation (6.14): compute one 3D point along each ray
    X = np.transpose(np.linalg.lstsq(
        camera[:,0:3],
        np.transpose(S)-np.expand_dims(camera[:,3],axis=1),
        rcond=None)[0])

    #   unit-vectors from camera center to each 3D point
    V = X - np.tile(C, (4, 1))
    V = V / np.linalg.norm(V, ord=2, axis=1, keepdims=True)

    # make sure these vectors point forwards from the camera instead of backwards
    V = V*np.expand_dims(np.sign(np.sum(V * np.tile(camera[2,0:3],(4, 1)), axis=1)),axis=1)

    #   desired ray segments that are length raysize in these directions
    V = np.tile(C, (4, 1)) + raysize * V

    # append the camera center itself to complete the four tetrahedral vertices
    V=np.vstack([C,V])

    # add camera center to figure
    figobj.add_trace(go.Scatter3d(
        x=[C[0]],
        y=[C[1]],
        z=[C[2]],
        mode='markers',
        marker=dict(
            size=3,
            color='#ff7f0e'
        )
    )
                    )


    # add tetrahedron to figure
    figobj.add_trace(go.Mesh3d(
        # vertices of tetrahedron
        x=V[:,0],
        y=V[:,1],
        z=V[:,2],

        # i, j and k give the vertices of triangles
        i=[0, 0, 0, 0],
        j=[1, 2, 3, 4],
        k=[2, 3, 4, 1],
        opacity=0.5,
        color='#ff7f0e'
    ))

    return 1

In [101]:
#@title `visualizeObjectPoints`
def visualizeObjectPoints(points, Ps):
  h, w, _ = im1.shape

  # Assuming 'points[:,3]' is a list of indices for colors
  # Here, 'colorscale' could be any Plotly supported scale like 'Viridis', 'Cividis', etc.
  fig = go.Figure()
  color_strings = [f"rgba({int(r)},{int(g)},{int(b)},1)" for r, g, b in points[:, -3:]]

  fig.add_trace(go.Scatter3d(
      x=points[:, 0],
      y=points[:, 1],
      z=points[:, 2],
      mode='markers',
      marker=dict(
            size=2,
            color=color_strings,  # Use color data directly from points array
        )
  ))

  # Add the cameras to the figure
  for i, P in enumerate(Ps):
      add_plotly_camera(h, w, P, 1, fig)

  # Update the layout for better visualization
  fig.update_layout(
      scene=dict(
          aspectmode='cube',
          xaxis=dict(title='X', range=[-30, 30]),
          yaxis=dict(title='Y', range=[-30, 30]),
          zaxis=dict(title='Z', range=[-30, 40])
      ),
      showlegend=False,
      scene_camera=dict(
          up=dict(x=0, y=-1, z=0),
          center=dict(x=0, y=0, z=0),
          eye=dict(x=0, y=-1, z=-1)
      )
  )

  fig.show()

In [102]:
total_3d_points_color = np.hstack([total_3d_points, total_colors])
visualizeObjectPoints(total_3d_points_color, cameras)

# COLMAP Comparison

Do not run unless necessary

In [ ]:
root_dir = Path('/content/drive/My Drive/es-143-assignments/Project/COLMAP-demo')

In [ ]:
# Location of all images to be reconstructed
image_dir = root_dir / 'playroom_lowres_images'

# Where to save the COLMAP outputs
colmap_dir = root_dir / 'colmap'
colmap_db_path = colmap_dir / 'database.db'
colmap_out_path = colmap_dir / 'sparse'

# Make all of these folders if they do not already exist
colmap_out_path.mkdir(exist_ok=True, parents=True)
image_dir.mkdir(exist_ok=True, parents=True)

print(f"""Directories configured:
  image_dir = {image_dir}
  colmap_dir = {colmap_dir}
  colmap_out_path = {colmap_out_path}
""")

Directories configured:
  image_dir = /content/drive/My Drive/es-143-assignments/Project/COLMAP-demo/playroom_lowres_images
  colmap_dir = /content/drive/My Drive/es-143-assignments/Project/COLMAP-demo/colmap
  colmap_out_path = /content/drive/My Drive/es-143-assignments/Project/COLMAP-demo/colmap/sparse



In [ ]:
#@title Install COLMAP (for reconstruction) and pycolmap (for visualizing output)
!apt-get install colmap
!pip install "git+https://github.com/google/nerfies.git#egg=pycolmap&subdirectory=third_party/pycolmap"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcamd2 libccolamd2 libceres2 libcholmod3 libcolamd2 libcxsparse3 libevdev2
  libfreeimage3 libgflags2.2 libgoogle-glog0v5 libgudev-1.0-0 libinput-bin libinput10 libjxr0
  libmd4c0 libmetis5 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5
  libqt5widgets5 libraw20 libspqr2 libsuitesparseconfig5 libwacom-bin libwacom-common libwacom9
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0
  libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0 qt5-gtk-platformtheme qttranslations5-l10n
Suggested packages:
  qt5-image-formats-plugins qtwayland5
The following NEW packages will be installed:
  colmap libamd2 libcamd2 libccolamd2 libceres2 libcholmod3 libcolamd2 libcxsparse3 libevdev2
  libfreeimage3 libgflags2.2 libgoogle-glog0v5 libgudev-1.0-0 libinput-bin libinput10 libjxr0


In [ ]:
# @title Configure camera settings and extract features

# This is where you specify whether the intrinsic parameters are shared
#   between cameras, and which intrinsic parameters should be estimated. This
#   demo shares intrinsic parameters across all cameras, and it uses the
#   SIMPLE_PINHOLE model, which has no radial distortion, no pixel skew, and the
#   same focal length for the x and y dimensions (i.e., "square pixels with zero skew")
#
#   See: https://colmap.github.io/cameras.html

# Releveant options are:
# --SiftExtraction.upright 1 \                 # turn off SIFT's orientation invariance
# --ImageReader.camera_model SIMPLE_PINHOLE \  # no radial distortion parameters
# --ImageReader.single_camera 1 \              # share intrinsics across all cameras

!colmap feature_extractor \
--SiftExtraction.use_gpu 0 \
--SiftExtraction.upright 1 \
--ImageReader.camera_model SIMPLE_PINHOLE \
--ImageReader.single_camera 1 \
--database_path "{str(colmap_db_path)}" \
--image_path "{str(image_dir)}"


Feature extraction

Processed file [1/24]
  Name:            playroom_es143_small_00001.JPG
  SKIP: Features for image already extracted.
Processed file [2/24]
  Name:            playroom_es143_small_00002.JPG
  SKIP: Features for image already extracted.
Processed file [3/24]
  Name:            playroom_es143_small_00003.JPG
  SKIP: Features for image already extracted.
Processed file [4/24]
  Name:            playroom_es143_small_00004.JPG
  SKIP: Features for image already extracted.
Processed file [5/24]
  Name:            playroom_es143_small_00005.JPG
  SKIP: Features for image already extracted.
Processed file [6/24]
  Name:            playroom_es143_small_00006.JPG
  SKIP: Features for image already extracted.
Processed file [7/24]
  Name:            playroom_es143_small_00007.JPG
  SKIP: Features for image already extracted.
Processed file [8/24]
  Name:            playroom_es143_small_00008.JPG
  SKIP: Features for image already extracted.
Processed file [9/24]
  Name:      

In [ ]:
# @title Match features

!colmap exhaustive_matcher \
--SiftMatching.use_gpu 0 \
--database_path "{str(colmap_db_path)}"


Exhaustive feature matching

Matching block [1/1, 1/1] in 0.011s
Elapsed time: 0.000 [minutes]


In [ ]:
# @title Reconstruct

# For description of all of the options, see https://github.com/mwtarnowski/colmap-parameters
# Some of the relevant ones are:
#  --Mapper.ba_refine_principal_point 1       # refine the principal points
#  --Mapper.min_num_matches 32                # minimum viable number of matches between a pair of images

!colmap mapper \
  --Mapper.ba_refine_principal_point 1 \
  --Mapper.filter_max_reproj_error 2 \
  --Mapper.tri_complete_max_reproj_error 2 \
  --Mapper.min_num_matches 32 \
  --database_path "{str(colmap_db_path)}" \
  --image_path "{str(image_dir)}" \
  --output_path "{str(colmap_out_path)}"


Loading database

Loading cameras... 1 in 0.000s
Loading matches... 226 in 0.009s
Loading images... 24 in 0.020s (connected 24)
Building correspondence graph... in 0.040s (ignored 19)

Elapsed time: 0.001 [minutes]


Finding good initial image pair


Initializing with image pair #17 and #20


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  6.912105e+03    0.00e+00    5.09e+05   0.00e+00   0.00e+00  1.00e+04        0    3.03e-03    7.42e-03
   1  3.312359e+02    6.58e+03    2.15e+05   3.01e+00   9.62e-01  3.00e+04        1    5.76e-03    1.32e-02
   2  7.446841e+01    2.57e+02    2.90e+04   1.17e+01   9.26e-01  7.85e+04        1    5.41e-03    1.87e-02
   3  6.360358e+01    1.09e+01    1.67e+04   1.14e+01   3.87e-01  7.76e+04        1    5.66e-03    2.44e-02
   4  4.569782e+01    1.79e+01    1.90e+03   3.62e+00   9.84e-01  2.33e+05        1    5.74e-03    3.02e-02
   5  4.527619e+01    4.22e-01 

In [ ]:
# @title Verify that the reconstruction succeeded

if not colmap_db_path.exists():
  raise RuntimeError(f'The COLMAP DB does not exist, did you run the reconstruction?')
elif not (colmap_dir / 'sparse/0/cameras.bin').exists():
  raise RuntimeError("""
SfM seems to have failed.
""")
else:
  print("Everything looks good!")

Everything looks good!


In [ ]:
#@title Use `pycolmap` to access COLMAPs output
import pycolmap
from pycolmap import Quaternion

# Initialize pycolmap scene manager
manager = pycolmap.SceneManager(str(colmap_dir / 'sparse/0'))

# load the model and discard points that do not survive across enough images
manager.load_cameras()
manager.load_images()
manager.load_points3D()
manager.filter_points3D(min_track_len=24)

# Get intrinsic parameters
cameras = manager.cameras     # intrinsic camera parameters (multiple elements if intrinsics were not shared)

# Get 3D points, a Nx3 numpy array
points3D = manager.points3D

# Get the per-image camera information.
#   This returns an ordered dictionary pycolmap.image objects.
#   Each object has attributes:
#   name        : image filename
#   camera_id   : index into cameras, which has the intrinsics
#   q           : camera orientation, a pycolmap.Quarternion object
#   t           : camera center as a list
#   t_vec       : camera center as a numpy array
#   points2D    : detected 2D feature points
#   points3D_ids: index into points3D for each element in points2D
images = manager.images

# Compose lists of camera centers and orientations
camera_centers = []
camera_orientations = []
for key, image in images.items():
    camera_centers.append(image.tvec)
    camera_orientations.append(image.q)

# Convert camera centers to a numpy array
camera_centers=np.array(camera_centers)

In [ ]:
#@title Simple interactive visualization

# To Do: Use the orientation and intrinsics information to draw a square-pyramid for each camera

import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=points3D[:, 0],
    y=points3D[:, 1],
    z=points3D[:, 2],
    mode='markers',
    marker=dict(size=2),
))
fig.add_trace(go.Scatter3d(
    x=camera_centers[:, 0],
    y=camera_centers[:, 1],
    z=camera_centers[:, 2],
    mode='markers',
    marker=dict(size=2),
))
fig.update_traces(showlegend=False)
fig.update_layout(scene_dragmode='orbit')
fig.update_layout(scene=dict(aspectmode='data'))

# adjust aspect ratio and initial viewing direction
fig.update_layout(showlegend=False,
                  scene_camera=dict(
                      up=dict(x=0, y=-1, z=0),
                      center=dict(x=0, y=0, z=0),
                      eye=dict(x=0, y=-1, z=-2)
                    ))
fig.show()